# Unification

This is actually Part I of a two part assignment. In a later module, you'll implement a Forward Planner. In order to do that, however, you need to have a unifier. It is important to note that you *only* need to implement a unifier. Although the module talked about resolution, you do not need to implement anything like "standardizing apart". From the unifier's point of view, that should already have been done.

Unification is simply the *syntactic* balancing of expressions. There are only 3 kinds of expressions: constants, lists and (logic) variables. Constants and lists are only equal to each other if they're exactly the same thing or can be made to be the same thing by *binding* a value to a variable.

It really is that simple...expressions must be literally the same (identical) except if one or the other (or both) has a variable in that "spot".

## S-Expressions

With that out of the way, we need a language with which to express our constants, variables and predicates and that language will be based on s-expressions.

**constants** - There are two types of constants, values and predicates. Values should start with an uppercase letter. Fred is a constant value, so is Barney and Food. Predicates are named using lowercase letters. loves is a predicate and so is hates. This is only a convention. Secret: your code does not need to treat these two types of constants differently.

**variables** - these are named using lowercase letters but always start with a question mark. ?x is a variable and so is ?yum. This is not a convention.

**expressions (lists)** - these use the S-expression syntax a la LISP. (loves Fred Wilma) is an expression as is (friend-of Barney Fred) and (loves ?x ?y).

## Parsing

These functions are already included in the starter .py file.

In [1]:
import tokenize
from io import StringIO

This uses the above libraries to build a Lisp structure based on atoms. It is adapted from [simple iterator parser](http://effbot.org/zone/simple-iterator-parser.htm). The first function is the `atom` function.

In [2]:
def atom( next, token):
    if token[ 1] == '(':
        out = []
        token = next()
        while token[ 1] != ')':
            out.append( atom( next, token))
            token = next()
            if token[ 1] == ' ':
                token = next()
        return out
    elif token[ 1] == '?':
        token = next()
        return "?" + token[ 1]
    else:
        return token[ 1]

The next function is the actual `parse` function:

In [3]:
def parse(exp):
    src = StringIO(exp).readline
    tokens = tokenize.generate_tokens(src)
    return atom(tokens.__next__, tokens.__next__())

**Note** there was a change between 2.7 and 3.0 that "hid" the next() function in the tokenizer.

From a Python perspective, we want to turn something like "(loves Fred ?x)" to ["loves" "Fred" "?x"] and then work with the second representation as a list of strings. The strings then have the syntactic meaning we gave them previously.

In [4]:
parse("Fred")

'Fred'

In [5]:
parse( "?x")

'?x'

In [6]:
parse( "(loves Fred ?x)")

['loves', 'Fred', '?x']

In [7]:
parse( "(father_of Barney (son_of Barney))")

['father_of', 'Barney', ['son_of', 'Barney']]

## Unifier

Now that that's out of the way, here is the imperative pseudocode for unification. This is a classic recursive program with a number of base cases. Students for some reason don't like it, try the algorithm in the book, can't get it to work and then come back to this pseudocode.

Work through the algorithm by hand with your Self-Check examples if you need to but I'd suggest sticking with this implementation. It does work.

Here is imperative pseudocode for the algorithm:

```
def unification( exp1, exp2):
    # base cases
    if exp1 and exp2 are constants or the empty list:
        if exp1 = exp2 then return {}
        else return FAIL
    if exp1 is a variable:
        if exp1 occurs in exp2 then return FAIL
        else return {exp1/exp2}
    if exp2 is a variable:
        if exp2 occurs in exp1 then return FAIL
        else return {exp2/exp1}

    # inductive step
    first1 = first element of exp1
    first2 = first element of exp2
    result1 = unification( first1, first2)
    if result1 = FAIL then return FAIL
    apply result1 to rest of exp1 and exp2
    result2 = unification( rest of exp1, rest of exp2)
    if result2 = FAIL then return FAIL
    return composition of result1 and result2
```

`unification` can return...

1. `None` (if unification completely fails)
2. `{}` (the empty substitution list) or 
3. a substitution list that has variables as keys and substituted values as values, like {"?x": "Fred"}. 

Note that the middle case sometimes confuses people..."Sam" unifying with "Sam" is not a failure so you return {} because there were no variables so there were no substitutions. You do not need to further resolve variables. If a variable resolves to an expression that contains a variable, you don't need to do the substition.

If you think of a typical database table, there is a column, row and value. This Tuple is a *relation* and in some uses of unification, the "thing" in the first spot..."love" above is called the relation. If you have a table of users with user_id, username and the value then the relation is:

`(login ?user_id ?username)`

*most* of the time, the relation name is specified. But it's not impossible for the relation name to be represented by a variable:

`(?relation 12345 "smooth_operator")`

Your code should handle this case (the pseudocode does handle this case so all  you have to do is not futz with it).

Our type system is very simple. We can get by with just a few boolean functions. The first tests to see if an expression is a variable.

In [8]:
def is_variable( exp):
    return isinstance( exp, str) and exp[ 0] == "?"

In [9]:
is_variable( "Fred")

False

In [10]:
is_variable( "?fred")

True

The second tests to see if an expression is a constant:

In [11]:
def is_constant( exp):
    return isinstance( exp, str) and not is_variable( exp)

In [12]:
is_constant( "Fred")

True

In [13]:
is_constant( "?fred")

False

In [14]:
is_constant( ["loves", "Fred", "?wife"])

False

It might also be useful to know that:

<code>
type( "a")
&lt;type 'str'>
type( "a") == str
True
type( "a") == list
False
type( ["a"]) == list
True
</code>


You need to write the `unification` function described above. It should work with two expressions of the type returned by `parse`. See `unify` for how it will be called. It should return the result of unification for the two expressions as detailed above and in the book. It does not have to make all the necessary substitions (for example, if ?y is bound to ?x and 1 is bound to ?y, ?x doesn't have to be replaced everywhere with 1. It's enough to return {"?x":"?y", "?y":1}. For an actual application, you would need to fix this!)

-----

<a id="return_value"></a>
## return_value

Returns the value from the string or list input format into the output format

* **value** str or list

**returns** str

In [15]:
def return_value(value:str)->str:
    if type(value) == list:
        left_value, right_value = value[0], value[1:]
        if type(right_value) == list:
            return f'{left_value}({return_value(right_value[0])})'
        else:
            return f'{0}({1})'. format(left_value, ','.join(right_value))
    else:
        return value

In [16]:
assert return_value("Fred") == "Fred"

assert return_value(["Fred", "Pebble"]) == "Fred(Pebble)"

assert return_value(["Loves", ["Fred", "Barney"]]) == "Loves(Fred(Barney))"

<a id="pair"></a>
## pair

pairs the variable string values for the next process in the algorithm

* **value** str

**returns** str

In [17]:
def pair(var_1, var_2):
    var = var_1[1:]
    val = return_value(var_2)
    return var, val

<a id="join"></a>
## join

joins the two variables into question into the desired output if the instance is a variable 

* **var_1** str
* **var_2** str
* **theta** str

**returns** list

In [18]:
def join(var_1:str, var_2:str, theta: list)->list:
    # Checks if occurs in the other
    if var_1 in var_2:
        return None
    else:
        var, value = pair(var_1, var_2)
        if var not in theta:
            theta.append(f'{var}/{value}')
        return theta 

In [19]:
test_var1 = '?x'
test_var2 = ['car', 'color']
test_theta = []

assert join(test_var1, test_var2, test_theta) == ['x/car(color)']
test_var3 = '?x'
test_var4 = ['car', 'model']
test_theta1 = []

assert join(test_var3, test_var4, test_theta1) == ['x/car(model)']
test_var5 = '?y'
test_var6 = 'model'
test_theta2 = []

assert join(test_var5, test_var6, test_theta2) == ['y/model']

<a id="to_dict"></a>
## to_dict

Helper function to convert the resulting list to a dictionary

* **results** list : the results

**returns** dict : list converted to dictionary

In [20]:
def to_dict(result):
    if result != None:
        x = []
        for i in range(len(result)):
            x += result[i].split('/')
        x = iter(x)
        result_dict = dict(zip(x,x))
        return result_dict
    else:
        return None

In [21]:
x = ['x/Pebbles']

assert to_dict(x) == {'x': 'Pebbles'}

y = ['x/Pebbles', 'y/Rocks']
assert to_dict(y) == {'x': 'Pebbles', 'y': 'Rocks'}

z = ['x/Pebbles', 'y/Rocks', 'z/Fred(Barney)']
assert to_dict(z) == {'x': 'Pebbles', 'y': 'Rocks', 'z': 'Fred(Barney)'}

<a id="unification"></a>
## unification

Unification algorithm that attempts to find subsitutions between two expressions that takes the form of {variable/constant}. Uniifcation is used for inference of first order logic. 

* **list_exrpession1** list : the first expression list
* **list_exrpession2** list : the second expression list
* **theta** list : the results

**returns** list

In [22]:
def unification(list_expression1:list, list_expression2:list, theta:list)->list:
    if theta is None:
        return theta
    elif type(list_expression1) == list and type(list_expression2) == list and len(list_expression1) == 0 and len(list_expression2) == 0:
        return theta if list_expression1 == list_expression2 else None
    elif is_constant(list_expression1) and is_constant(list_expression2):
        return theta if list_expression1 == list_expression2 else None
    elif is_variable(list_expression1):
        return join(list_expression1, list_expression2, theta)
    elif is_variable(list_expression2):
        return join(list_expression2, list_expression1, theta)
    
    result_1 = unify(list_expression1[0], list_expression2[0])
    if result_1 is None: return None
    
    result_2 = unify(list_expression1[1:], list_expression2[1:])
    if result_2 is None: return None
    return result_1 + result_2 

<a id="unify"></a>
## unify
Helper function to initate the unification algorithm
* **s_exrpession1** list : the first expression list
* **s_exrpession2** list : the second expression list

**returns** fucntion execution of unification

In [23]:
def unify(s_expression1, s_expression2):
    theta = list()
    return unification(s_expression1, s_expression2, theta)

**Note** If you see the error,

```
tokenize.TokenError: ('EOF in multi-line statement', (2, 0))
```
You most likely have unbalanced parentheses in your s-expression.

## Test Cases

Use the expressions from the Self Check as your test cases...

In [24]:
self_check_test_cases = [
    ['(son Barney Barney)', '(daughter Wilma Pebbles)', None],
    ['Fred', 'Barney', None],
    ['Pebbles', 'Pebbles', {}],
    ['(quarry_worker Fred)', '(quarry_worker ?x)', {'x': 'Fred'}],
    ['(son Barney ?x)', '(son ?y Bam_Bam)', {'y': 'Barney', 'x': 'Bam_Bam'}],
    ['(married ?x ?y)', '(married Barney Wilma)', {'x': 'Barney', 'y': 'Wilma'}],
    ['(son Barney ?x)', '(son ?y (son Barney))', {'y': 'Barney', 'x': 'son(Barney)'}],
    ['(son Barney ?x)', '(son ?y (son ?y))',  {'y': 'Barney', 'x': 'son(?y)'}],
    ['(son Barney Bam_Bam)', '(son ?y (son Barney))',  None],
    ['(loves Fred Fred)', '(loves ?x ?x)', {'x': 'Fred'}],
    ['(future George Fred)', '(Future ?y ?y)', None]
]
for case in self_check_test_cases:
    exp1, exp2, expected = case
    actual = to_dict(unify(parse(exp1), parse(exp2)))
    print(f"Expression 1 = {parse(exp1)}, Expression 2 = {parse(exp2)}")
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

Expression 1 = ['son', 'Barney', 'Barney'], Expression 2 = ['daughter', 'Wilma', 'Pebbles']
actual = None
expected = None


Expression 1 = Fred, Expression 2 = Barney
actual = None
expected = None


Expression 1 = Pebbles, Expression 2 = Pebbles
actual = {}
expected = {}


Expression 1 = ['quarry_worker', 'Fred'], Expression 2 = ['quarry_worker', '?x']
actual = {'x': 'Fred'}
expected = {'x': 'Fred'}


Expression 1 = ['son', 'Barney', '?x'], Expression 2 = ['son', '?y', 'Bam_Bam']
actual = {'y': 'Barney', 'x': 'Bam_Bam'}
expected = {'y': 'Barney', 'x': 'Bam_Bam'}


Expression 1 = ['married', '?x', '?y'], Expression 2 = ['married', 'Barney', 'Wilma']
actual = {'x': 'Barney', 'y': 'Wilma'}
expected = {'x': 'Barney', 'y': 'Wilma'}


Expression 1 = ['son', 'Barney', '?x'], Expression 2 = ['son', '?y', ['son', 'Barney']]
actual = {'y': 'Barney', 'x': 'son(Barney)'}
expected = {'y': 'Barney', 'x': 'son(Barney)'}


Expression 1 = ['son', 'Barney', '?x'], Expression 2 = ['son', '?y', ['son', '?

Now add at least **five (5)** additional test cases of your own making, explaining exactly what you are testing. They should not be testing the same things as the self check test cases above.

In [25]:
new_test_cases = [
    ['(son Barney Barney)', '(daughter Wilma Pebbles)', None, "non-equal constants"], 
    ['(son Fred Barney)', '(daughter Fred ?x)', None, "non-eqaul constants" ],
    ['(Barney ?x ?x)', '(Barney Fred Wilam)', {'x': 'Wilam'}, "assigning two values to same variable"], 
    ['(Dog Cat)', '(Dog ?y)', {'y': 'Cat'}, 'Testing for Cat and z unification'],
    ['(Car ?x ?y ?z)', '(Car Make Color Price)', {'x': 'Make', 'y': 'Color', 'z': 'Price'}, 'Testing for 3 Variables'],
    ['(Car Color ?x)', '(Car Color ?y)', {'x': '?y'}, 'Testing variable to variable']
]
for case in new_test_cases:
    exp1, exp2, expected, message = case
    actual = to_dict(unify(parse(exp1), parse(exp2)))
    print(f"Testing {message}...")
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

Testing non-equal constants...
actual = None
expected = None


Testing non-eqaul constants...
actual = None
expected = None


Testing assigning two values to same variable...
actual = {'x': 'Wilam'}
expected = {'x': 'Wilam'}


Testing Testing for Cat and z unification...
actual = {'y': 'Cat'}
expected = {'y': 'Cat'}


Testing Testing for 3 Variables...
actual = {'x': 'Make', 'y': 'Color', 'z': 'Price'}
expected = {'x': 'Make', 'y': 'Color', 'z': 'Price'}


Testing Testing variable to variable...
actual = {'x': '?y'}
expected = {'x': '?y'}


